In [ ]:
!pip install controlnet_aux
!pip install diffusers transformers accelerate

In [ ]:
import torch
import os
from huggingface_hub import HfApi
from pathlib import Path
from diffusers.utils import load_image
from PIL import Image
import numpy as np
from controlnet_aux import OpenposeDetector

from diffusers import (
    ControlNetModel,
    StableDiffusionControlNetPipeline,
    UniPCMultistepScheduler,
)

checkpoint = "lllyasviel/control_v11p_sd15_openpose"


In [4]:
image = load_image("") ##load original

processor = OpenposeDetector.from_pretrained('lllyasviel/ControlNet')

control_image = processor(image, hand_and_face=True)
control_image.save("/content/control.png")


In [ ]:
from IPython.display import Image
image_path = '/content/control.png'
Image(filename=image_path)

In [ ]:
controlnet = ControlNetModel.from_pretrained(checkpoint, torch_dtype=torch.float16)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
generator = torch.manual_seed(0)


In [ ]:
prompt = "" ##prompt here
quality = 10 #num
image = pipe(prompt, num_inference_steps=quality, generator=generator, image=control_image).images[0]
image.save('/content/image_out.png')

In [9]:
#show output
from IPython.display import Image

image_path = '/content/SAM_2264.png'

Image(filename=image_path)
